In [13]:
import cv2
import numpy as np
import torch
import torch.nn as nn
from torchvision.transforms import ToTensor
import os

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
#print(device)

os.environ['CUDA_LAUNCH_BLOCKING']='1'
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer_1 = nn.Sequential(
          nn.Conv2d(1, 16, 3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2))
        self.conv_layer_2 = nn.Sequential(
          nn.Conv2d(16, 32, 3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout(0.2))
        self.classifier = nn.Sequential(
          nn.Flatten(),
          nn.Linear(in_features=32*30*30, out_features=128),
          nn.ReLU(),
          nn.Dropout(0.05),
          nn.Linear(in_features=128, out_features=5))
    def forward(self, x: torch.Tensor):
        x = self.conv_layer_1(x)
        x = self.conv_layer_2(x)
        x = self.classifier(x)
        return x
    
def define_module():
    global model
    model = ImageClassifier()
    model.to(device)
    model.load_state_dict(torch.load("./weight/black0-4(99).pth"))
    model.eval()
    
def totensor(result_image):
    to_tensor = ToTensor()
    tensor_image = to_tensor(result_image)
    tensor_image = tensor_image.unsqueeze(0)
    #tensor_image.shape
    return tensor_image

def predict(tensor_image):
    out = model(tensor_image.to(device))
    soft=nn.Softmax(dim=1)
    out = soft(out)
    print(out)
    max_value, max_index = torch.max(out, dim=1)
    print("predict number", max_index.item())
    if(max_index.item()<0.5):
        return -1
    return max_index.item()
    
def seg(image,saturation_threshold = 70):
    hsi= cv2.cvtColor(image, cv2.COLOR_RGB2HSV) 
    h, s, i = cv2.split(hsi)    
    saturation_mask = s < saturation_threshold
    saturation_mask = saturation_mask.astype(np.uint8)
    saturation_mask = 1- saturation_mask 
    saturation_mask = saturation_mask * 255
    return saturation_mask

def all_process(image):
    define_module()
    saturation_mask = seg(image)
    resize_image = cv2.resize(saturation_mask,(128,128),interpolation=cv2.INTER_AREA)
    tensor_image = totensor(resize_image)
    predict_number = predict(tensor_image)
    return predict_number

In [15]:
figure0 =cv2.imread('./zerotofour/3/three100.jpg')

predict_number = all_process(figure0)
predict_number
# -1 代表機率小於0.5、0,1,2,3,4,5代表各自預測結果

tensor([[3.0657e-06, 1.0563e-11, 9.5005e-06, 9.9998e-01, 1.1906e-05]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)
predict number 3


3